# This notebook solves the Huggett Model (1993) using Dolo.

This is a heterogeneous-agent model where each period agents are hit with idiosyncratic income shocks $y_t$ that follow an $AR1$ process. There are incomplete markets and agents only have access to a risk-free asset $s_t$ that pays $(1+r)s_t$ next period, where $r$ is the interest rate.

The value function for an agent with current assets $s$ and current income $y$ is: $v(y,s)=\max_{c,s'} u(c)+\beta \mathbf{E}v(y',s')$ where the expectation is taken over the value of the income shock.

The agent's budget constraint is: $c+s'=(1+r)s+y$ where s' is his asset choice next period. The agent will also be subject to a borrowing constraint: $s'\geq \bar{s}$.

Here, we define the control in the model as $a=s'-s$, i.e. $a$ is the change in assets.


In [1]:
# First import the packages
Pkg.dir("Dolo")
import Dolo
using AxisArrays
using PyPlot

. To use SymEngine run the following code: `Pkg.add("SymEngine")`

In [2]:
# get the model file
filename=("huggett_1993.yaml")

(Dolo.DiscreteMarkovProcess) in module Dolo at C:\Users\Angela\AppData\Local\JuliaPro-0.5.2.2\pkgs-0.5.2.2\v0.5\Dolo\src\numeric/processes.jl:37 overwritten

"huggett_1993.yaml"

In [3]:
# Convert the file into Dolo model
model=Dolo.yaml_import(filename)

 at C:\Users\Angela\AppData\Local\JuliaPro-0.5.2.2\pkgs-0.5.2.2\v0.5\Dolo\src\numeric/processes.jl:157.


name,Huggett 1993
filename,huggett_1993.yaml
Type,Equation
value,\[V_{t} = \frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}+\beta V_{t+1}\]
expectation,\[m_{t} = \frac{\beta}{\left(c_{t+1}\right)^{\sigma}} 1+r\]
felicity,\[u_{t} = \frac{\left(c_{t}\right)^{\left(1-\sigma\right)}}{\left(1-\sigma\right)}\]
transition,\[s_{t} = a_{t-1}+s_{t-1}\]
arbitrage,\[\left(1-\beta \left(\frac{c_{t}}{c_{t+1}}\right)^{\sigma} 1+r\right)\]


Model


Create a function to loop over guesses of the interest rate until the asset market clears.

Now let's look at solving the model. We will use Dolo's time iteration function (which iterates on the residuals of the arbitrage equation).

In [4]:
@time sol=Dolo.time_iteration(model,verbose=true, maxit=1000)
dr=sol.dr


------------------------------------------------------------------
It    ηₙ=|xₙ-xₙ₋₁|    λₙ=ηₙ/ηₙ₋₁      Time            Newton steps
------------------------------------------------------------------
0     NaN             NaN             0.00e+00        0    
1     1.42e+00        NaN             1.32e+00        5    
2     8.12e-01        5.71e-01        3.25e-02        4    
3     4.94e-01        6.08e-01        2.95e-02        4    
4     3.19e-01        6.46e-01        3.27e-02        4    
5     2.16e-01        6.79e-01        2.48e-02        3    
6     1.53e-01        7.05e-01        2.87e-02        3    
7     1.11e-01        7.27e-01        1.99e-02        3    
8     8.27e-02        7.45e-01        2.14e-02        3    
9     6.29e-02        7.61e-01        3.15e-02        3    
10    4.87e-02        7.74e-01        2.17e-02        3    
11    3.82e-02        7.85e-01        2.10e-02        3    
12    3.04e-02        7.95e-01        2.31e-02        3    
13    2.54e-02     

136   4.55e-03        9.24e-01        1.69e-02        2    
137   4.21e-03        9.24e-01        1.99e-02        2    
138   3.88e-03        9.23e-01        1.90e-02        2    
139   3.58e-03        9.23e-01        1.46e-02        2    
140   3.31e-03        9.23e-01        1.79e-02        2    
141   3.05e-03        9.22e-01        1.51e-02        2    
142   2.81e-03        9.22e-01        1.69e-02        2    
143   2.59e-03        9.22e-01        1.58e-02        2    
144   2.39e-03        9.21e-01        1.77e-02        2    
145   2.20e-03        9.21e-01        1.39e-02        2    
146   2.03e-03        9.21e-01        8.27e-03        1    
147   1.87e-03        9.21e-01        9.19e-03        1    
148   1.72e-03        9.21e-01        1.12e-02        1    
149   1.58e-03        9.20e-01        1.08e-02        1    
150   1.46e-03        9.20e-01        1.25e-02        1    
151   1.34e-03        9.20e-01        8.46e-03        1    
152   1.23e-03        9.20e-01        8.

Dolo tabulate gives us the decision rules.

In [5]:
drtab = Dolo.tabulate(model, dr, :s) 

Dolo.DecisionRule{Dolo.CartesianGrid,Dolo.CartesianGrid}


2-dimensional AxisArray{Float64,2,...} with axes:
    :V, Symbol[:lny,:s,:a]
    :s, [-2.0,-1.77778,-1.55556,-1.33333,-1.11111,-0.888889,-0.666667,-0.444444,-0.222222,0.0  …  18.0,18.2222,18.4444,18.6667,18.8889,19.1111,19.3333,19.5556,19.7778,20.0]
And data, a 3×100 Array{Float64,2}:
  0.0        0.0        0.0        0.0      …   0.0        0.0        0.0    
 -2.0       -1.77778   -1.55556   -1.33333     19.5556    19.7778    20.0    
  0.271821   0.241945   0.212451   0.18372     -0.897489  -0.905228  -0.91296

Now we plot the consumption policy function. We see that it is concave because of the precautionary savings motive noting as well that there is more curvature closer to the borrowing constraint.

In [ ]:
# Plot the consumption policy function
import PyPlot
plt=PyPlot
r=model.calibration.flat[:r]
print(r)
c=exp(drtab[Axis{:V}(:lny)])+drtab[:s]*r-drtab[Axis{:V}(:a)]

plt.plot(drtab[Axis{:V}(:s)],c, color="blue")
plt.xlabel("Savings")
plt.ylabel("Consumption")
plt.title("Consumption Policy Function")

### Simulations

Here we run simulations for N agents and look at the asset distribution.

In [ ]:
# Simulations
import PyPlot
plt=PyPlot

N=500
T=500

mc_ar = Dolo.discretize(Dolo.MarkovChain, model.exogenous; N=5)
Index_mc=Dolo.simulate(mc_ar, 500, 500, 5)
@time sim_armc = Dolo.simulate(model,dr, Index_mc, mc_ar)

  2.530021 seconds (1.97 M allocations: 2.018 GB, 34.55% gc time)


3-dimensional AxisArray{Float64,3,...} with axes:
    :N, [1,2,3,4,5,6,7,8,9,10  …  491,492,493,494,495,496,497,498,499,500]
    :V, Symbol[:mc_process,:lny,:s,:a,:c,:y]
    :T, [1,2,3,4,5,6,7,8,9,10  …  491,492,493,494,495,496,497,498,499,500]
And data, a 500×6×500 Array{Float64,3}:
[:, :, 1] =
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 5.0  1.51865  0.0  13.4601  -8.89404  4.56607
 ⋮                                     ⋮      
 5.0  1.51865  0.0  13.4601  -8.89404  4.5660

In [18]:

#sim_mc = mc_ar.values[Index_mc]


2-dimensional AxisArray{Int64,2,...} with axes:
    :T, 1:500
    :N, 1:500
And data, a 500×500 Array{Int64,2}:
 5  5  5  5  5  5  5  5  5  5  5  5  5  …  5  5  5  5  5  5  5  5  5  5  5  5
 5  5  5  4  5  4  5  4  5  5  5  5  5     5  5  5  5  5  4  5  5  3  4  5  5
 5  5  5  4  4  4  5  4  5  3  5  5  5     5  5  5  5  5  4  4  5  3  4  4  5
 5  5  5  4  4  4  5  4  4  3  5  4  4     4  5  4  5  5  3  4  5  2  3  4  4
 5  5  5  4  4  4  5  3  4  2  5  4  4     4  4  4  5  4  3  5  5  2  3  4  4
 5  5  5  3  4  4  5  3  4  2  5  4  4  …  4  4  3  5  4  3  5  4  2  3  4  4
 5  4  5  2  5  4  5  3  4  2  5  4  4     3  4  3  4  4  4  4  4  2  3  5  5
 5  4  5  2  5  3  5  4  4  2  5  3  4     3  3  3  4  3  4  5  4  2  3  5  5
 4  4  5  2  5  2  5  4  4  2  3  3  4     4  3  3  4  3  4  5  5  2  3  5  4
 4  4  5  2  4  2  5  4  4  2  3  3  4     4  4  3  4  3  4  4  5  2  3  4  4
 3  4  3  2  3  2  5  3  4  2  3  3  4  …  4  4  3  4  2  4  4  4  3  3  4  4
 3  5  3  3  3  3  4  3  4  3 

In [ ]:
@time sim_armc = Dolo.simulate(model, dr,mc_ar)


In [ ]:
N=1000
n=200 # number of periods to plot
hor=linspace(1,n,n)
function plot_simulations(N::Int64,T::Int64,n::Int64,sim_armc)
    assets_end=zeros(N)
    for ii=1:N # number of simulations
          
        c=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T-n+1:T]+sim_armc[Axis{:N}(ii), Axis{:V}(:s)][T-n+1:T]*r-sim_armc[Axis{:N}(ii), Axis{:V}(:a)][T-n+1:T]
        assets_end[ii]=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T]-c[end]

        #assets_end[ii]=exp(sim_armc[Axis{:N}(ii), Axis{:V}(:lny)])[T]-c[end]
    
    end

    return assets_end
end




In [ ]:
assets_end=plot_simulations(N,T,n,sim_armc);

We see that the average asset holdings is very close to zero so we have market clearing.

In [ ]:
using Plots
histogram(assets_end,nbins=80,xlims=(-2.0,4.0), ylims=(0.0,120.0), label="", xlabel="Assets", ylabel="Frequency", title="Assets Distribution")


In [ ]:
mean(assets_end)